In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch.nn as nn
from embedding import CustomEmbedding
from transformer import EncoderDecoderTransformer
from utils import padding_collate_fn
import numpy as np

from generate_data import RandomIntegerDataset

In [24]:
n_real_tokens = 10
PAD_TOKEN_IDX = n_real_tokens
SOS_TOKEN_IDX = n_real_tokens + 1
EOS_TOKEN_IDX = n_real_tokens + 2
vocab_size = n_real_tokens + 3
D_MODEL = 64

embeddings = CustomEmbedding(vocab_size, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[ 1.0170, -1.8612,  0.0949, -1.4455, -0.9831,  0.8763,  0.9765,  1.4645,
         -0.4792, -0.3254, -0.3031, -1.6035,  0.0319,  0.5670, -0.1774,  0.0571,
          0.0234, -0.3045, -0.3482, -1.1616, -1.2573, -0.0209, -0.5584, -0.6112,
         -0.8776, -1.4018, -0.3302,  1.2415,  0.8192,  0.6834,  0.1408,  0.0175,
          1.8761,  1.7228, -0.8939,  0.5002, -0.5970, -0.8103, -0.7390,  0.7757,
         -0.2005, -0.7578, -0.8299, -1.5604,  0.5300,  2.3376,  0.0438,  0.6962,
         -1.4133,  1.1055, -1.5579,  0.1802,  1.8110, -1.2391,  0.7457,  0.8800,
          1.7837,  0.7143,  0.1310,  0.7310,  0.1752,  0.3385, -1.3181,  1.0158],
        [ 1.0971, -1.1075,  0.1615, -0.0286,  0.7733, -0.2958, -0.5305, -0.7767,
          1.0626,  2.2889,  0.0642,  1.0136, -0.5134,  1.7122,  1.1272,  0.2059,
          0.9449, -0.9393, -1.7916, -0.8560,  0.9218,  1.4515, -0.0310, -0.8140,
         -1.1044, -1.2348,  0.1666, -1.0685, -0.5922, -0.1658,  0.3266,  1.2131,
          1.0153,  0.3292, 

In [25]:
MAX_CONTEXT_WINDOW = 50

BATCH_SIZE = 64
MIN_SEQ_LEN = 2
MAX_SEQ_LEN = min(20, MAX_CONTEXT_WINDOW)

NUM_TRAINING_SEQUENCES = 10000
NUM_VALIDATION_SEQUENCES = 1000

VOCAB = [i for i in range(n_real_tokens)] # does not include SOS, EOS, PAD

VOCAB_MAP = dict()

for i, token in enumerate(VOCAB):
    VOCAB_MAP[i] = token
VOCAB_MAP[len(VOCAB_MAP)] = '<PAD>'
VOCAB_MAP[len(VOCAB_MAP) + 1] = '<SOS>'
VOCAB_MAP[len(VOCAB_MAP) + 2] = '<EOS>'

train_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_TRAINING_SEQUENCES, VOCAB)
train_dataloader = DataLoader(train_rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

val_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_VALIDATION_SEQUENCES, VOCAB)
val_dataloader = DataLoader(val_rand_ds, batch_size = BATCH_SIZE, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [26]:
input, label = next(iter(train_dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[ 3,  8,  2,  ...,  8,  1,  7],
        [ 2,  4, 10,  ..., 10, 10, 10],
        [ 1,  7,  3,  ..., 10, 10, 10],
        ...,
        [ 7,  6,  2,  ...,  0,  5,  3],
        [ 9,  3,  3,  ...,  9,  8,  9],
        [ 6,  4,  1,  ..., 10, 10, 10]])
tensor([[11,  0,  0,  ...,  7,  8,  8],
        [11,  2,  4,  ..., 10, 10, 10],
        [11,  1,  1,  ..., 10, 10, 10],
        ...,
        [11,  0,  0,  ...,  8,  8,  9],
        [11,  1,  1,  ...,  9,  9,  9],
        [11,  1,  4,  ..., 10, 10, 10]])
tensor([[ 0,  0,  0,  ...,  8,  8, 12],
        [ 2,  4, 12,  ..., 10, 10, 10],
        [ 1,  1,  3,  ..., 10, 10, 10],
        ...,
        [ 0,  0,  0,  ...,  8,  9, 12],
        [ 1,  1,  2,  ...,  9,  9, 12],
        [ 1,  4,  6,  ..., 10, 10, 10]])


In [27]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX, reduction = 'sum')

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 2, 
                    num_decoder_layers = 2, 
                    dim_feedforward = 32, 
                    dropout = 0.0,
                    max_context_window = MAX_CONTEXT_WINDOW,
                    use_pre_lnorm = True)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-4, momentum = 0.9, weight_decay = 1e-4)

In [28]:
def greedy_decode(source: torch.Tensor, model: nn.Module) -> torch.Tensor:
    """
    Designed to do autoregressive inference in an Encoder-Decoder transformer.

    This greedy decoder always predicts the vocabulary token corresponding to the highest logit.

    Takes the source sequence and the Encoder-Decoder to produce a predicted
    sequence starting from <SOS>.

    Note: This function *can* handle batches of sequences.

    Args:
        source - The source sequence to be passed to the Transformer's encoder block.
        model - The Encoder-Decoder transformer over which to greedy decode

    Returns:
        target - The batch of predicted sequences corresponding to the input sources.
        target_logits - (# batch elements =) batch_size (# rows =) seq_len (# cols =) vocab-dimensional vectors, each of which
        corresponds to the set of logits on a particular inference step within a given sequence.
    """
    batch_size = source.size(dim = 0)

    encoder_output, source_pad_mask = model.encode(source)

    # target will contain num_batch sequences of indices that are the predicted next-words for each batch element
    target = torch.full((batch_size, 1), SOS_TOKEN_IDX) # target.shape: [batch_size, num_loops_complete - 1]
    target_logits = torch.zeros((batch_size, 1, vocab_size))

    finished = torch.full((batch_size, ), False)

    while not finished.all() and target.size(dim = 1) <= MAX_CONTEXT_WINDOW:

        decoder_output, _ = model.decode(target, encoder_output, source_pad_mask)
        pred_logits = model.project_into_vocab(decoder_output) # pred_logits.shape: [batch_size, seq_len, vocab_size]

        last_row_pred_logits = pred_logits[:, -1, :] # last_row_pred_logits.shape == [batch_size, vocab_size]

        # Track next-word logits for loss_fn later.
        target_logits = torch.concat((target_logits, last_row_pred_logits.unsqueeze(1)), dim = 1)

        predictions = torch.argmax(last_row_pred_logits, dim = -1) # predictions.shape: [batch_size]

        # For any finished sequences (i.e. previous EOS-producers), force their prediction from this round to be a pad.
        predictions[finished] = PAD_TOKEN_IDX

        # Mark any additional sequences that just produced an EOS as finished.
        finished |= predictions == EOS_TOKEN_IDX

        target = torch.concat((target, predictions.reshape(-1, 1)), dim = 1) # target.shape: [batch_size, num_loops_complete]

    return target, target_logits[:, 1:, :]

In [29]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    Runs one training epoch (processing the entire training dataset once).
    
    Uses Teacher Forcing to train token-to-token mapping quality without cascading errors and for parallelization.

    Args:
        dataloader - The dataloader to process the dataset in BATCH_SIZE batches
        model - The Encoder-Decoder that is being trained
        loss_fn - The loss function to calculate the model's correctness
        optimizer - The optimizer to improve the model's weights
        calculate_sequence_accuracy - A flag to mark whether sequence-level correctness should be tracked
        calculate_token_accuracy - A flag to mark whether token-level correctness should be tracked
    """
    model.train()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    for (source, target), label in tqdm(dataloader):

        # FORWARD
        pred_logits = model(source, target)

        # pred_logits.shape: [batch_size, seq_len, vocab_size]
        # label.shape: [batch_size, seq_len]

        # CrossEntropyLoss (loss_fn) only takes 2D predictions (n_batch * seq_len, vocab_size) and 1D labels (n_batch * seq_len)
        batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

        # LOG
        with torch.no_grad():
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

        # BACKWARD
        batch_loss.backward()

        # OPTIMIZE
        optimizer.step()
        optimizer.zero_grad()

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_gold_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    Runs one validation epoch (processing the entire validation dataset once). 

    Uses Teacher Forcing (i.e. "gold") to evaluate token-to-token mapping quality and for parallelization.

    Args:
        dataloader - The dataloader to process the dataset in BATCH_SIZE batches
        model - The Encoder-Decoder that is being trained
        loss_fn - The loss function to calculate the model's correctness
        calculate_sequence_accuracy - A flag to mark whether sequence-level correctness should be tracked
        calculate_token_accuracy - A flag to mark whether token-level correctness should be tracked
    """
    model.eval()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    with torch.no_grad():
        
        for (source, target), label in tqdm(dataloader):
            
            # FORWARD
            pred_logits = model(source, target)
            batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

            # LOG
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_autoregressive_validation_loop(dataloader: DataLoader, model: nn.Module):
    """
    Runs one autoregressive validation epoch (processing the entire validation dataset once). 

    Args:
        dataloader - The dataloader to process the dataset in BATCH_SIZE batches
        model - The Encoder-Decoder that is being trained
    """
    model.eval()

    correct_sequences = 0
    incorrect_sequences = 0
    total_sequences = 0

    with torch.no_grad():
        
        for (source, _), label in tqdm(dataloader):

            # FORWARD
            pred_indices, pred_logits = greedy_decode(source, model)

            np_source_indices = source.numpy().copy()
            np_pred_target_indices = pred_indices.numpy().copy()

            token_values = np.array(list(VOCAB_MAP.values()))
            predicted_source_tokens = token_values[np_source_indices]
            predicted_target_tokens = token_values[np_pred_target_indices]

            for s, t in zip(predicted_source_tokens, predicted_target_tokens):
                source_end_index = np.argmax(s == '<PAD>')  if '<PAD>' in s else len(s)
                target_end_index = np.argmax(t == '<EOS>')
                if np.array_equal(np.sort(s[:source_end_index]), t[1:target_end_index]):
                    correct_sequences += 1
                else:
                    incorrect_sequences += 1
                    print(f'Incorrect Sequence {incorrect_sequences}:')
                    print(np.sort(s[:source_end_index]))
                    print(t[1:target_end_index])
                    print(f'{'Source:':<20} {s}\n{'Predicted Target:':<20} {t}', end = '\n\n')

            total_sequences += predicted_target_tokens.shape[0]

    return correct_sequences / total_sequences


In [30]:
EPOCHS = 10

training_losses = list()
training_sequence_accuracies = list()
training_token_accuracies = list()

gold_validation_losses = list()
gold_validation_sequence_accuracies = list()
gold_validation_token_accuracies = list()

for i in range(EPOCHS):
    # print(f'Running epoch {i+1}...')

    training_loss, training_sequence_accuracy, training_token_accuracy = run_train_epoch(train_dataloader, model, loss_fn, optim, calculate_sequence_accuracy = True, calculate_token_accuracy = True)

    training_losses.append(training_loss)
    training_sequence_accuracies.append(training_sequence_accuracy)
    training_token_accuracies.append(training_token_accuracy)

    gold_val_loss, gold_val_sequence_accuracy, gold_val_token_accuracy = run_gold_validation_loop(val_dataloader, model, loss_fn, calculate_sequence_accuracy = True, calculate_token_accuracy = True)
    
    gold_validation_losses.append(gold_val_loss)
    gold_validation_sequence_accuracies.append(gold_val_sequence_accuracy)
    gold_validation_token_accuracies.append(gold_val_token_accuracy)

print(training_losses)
print(training_sequence_accuracies)
print(training_token_accuracies)

print()

print(gold_validation_losses)
print(gold_validation_sequence_accuracies)
print(gold_validation_token_accuracies)

100%|██████████| 16/16 [00:00<00:00, 54.97it/s]

[6.258787748908997, 1.610886739730835, 1.2090406642913818, 0.6006722292900085, 0.7355378001689911, 0.5878827407360077, 0.40480144998431206, 0.03494530733823776, 0.016124046391248703, 0.012587716564536095]
[0.0019, 0.0143, 0.0243, 0.0381, 0.0364, 0.04, 0.0459, 0.058, 0.0573, 0.0573]
[0.4575564325478533, 0.5474433768016472, 0.5561056262864984, 0.5659671880961465, 0.5639254385964912, 0.5665359514540875, 0.5690154808205597, 0.5738731696156193, 0.5737783029656172, 0.5737982235437633]

[2.5884517669677733, 1.8381491737365723, 2.138014938354492, 0.7837490291595459, 1.789077018737793, 2.329984046936035, 0.04842299807071686, 0.04263178741931915, 0.012872815608978272, 0.25417310190200804]
[0.007, 0.002, 0.007, 0.018, 0.003, 0.011, 0.056, 0.054, 0.056, 0.05]
[0.5164310278945358, 0.532623232709209, 0.5316201757737867, 0.5542128391287734, 0.534581581964081, 0.5295662972869698, 0.565628582346198, 0.5653897592663355, 0.5658196408100878, 0.5621895299961789]


In [31]:
acc = run_autoregressive_validation_loop(val_dataloader, model)
print(acc)

  0%|          | 0/16 [00:00<?, ?it/s]

Incorrect Sequence 1:
['1' '1' '2' '2' '3' '3' '3' '3' '3' '4' '4' '5' '7' '7' '8' '9']
['1' '1' '2' '2' '3' '3' '3' '3' '3' '3' '4' '4' '5' '7' '7' '8' '9']
Source:              ['7' '4' '9' '8' '2' '1' '1' '7' '3' '5' '3' '2' '3' '3' '4' '3' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '1' '1' '2' '2' '3' '3' '3' '3' '3' '3' '4' '4' '5' '7' '7' '8'
 '9' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']

Incorrect Sequence 2:
['2' '2' '3' '3' '5' '6' '8']
['2' '2' '3' '3' '3' '5' '6' '8']
Source:              ['2' '6' '3' '5' '8' '3' '2' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '2' '2' '3' '3' '3' '5' '6' '8' '<EOS>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>']



 12%|█▎        | 2/16 [00:00<00:02,  5.78it/s]

Incorrect Sequence 3:
['1' '1' '1' '2' '3' '3' '3' '3' '3' '5' '5' '6' '7' '8' '9' '9' '9' '9'
 '9']
['1' '1' '1' '2' '3' '3' '3' '3' '3' '3' '5' '5' '6' '7' '8' '9' '9' '9'
 '9' '9']
Source:              ['2' '9' '3' '1' '9' '8' '3' '3' '3' '9' '7' '9' '6' '1' '1' '5' '9' '3'
 '5' '<PAD>']
Predicted Target:    ['<SOS>' '1' '1' '1' '2' '3' '3' '3' '3' '3' '3' '5' '5' '6' '7' '8' '9'
 '9' '9' '9' '9' '<EOS>' '<PAD>']

Incorrect Sequence 4:
['0' '3' '3' '3' '4' '4' '4' '5' '5' '6' '6' '6' '6' '7' '7' '7' '7' '7'
 '8' '9']
['0' '3' '3' '3' '3' '4' '4' '4' '5' '5' '6' '6' '6' '6' '7' '7' '7' '7'
 '7' '8' '9']
Source:              ['5' '6' '6' '6' '8' '7' '3' '7' '4' '3' '7' '7' '4' '3' '6' '4' '7' '9'
 '5' '0']
Predicted Target:    ['<SOS>' '0' '3' '3' '3' '3' '4' '4' '4' '5' '5' '6' '6' '6' '6' '7' '7'
 '7' '7' '7' '8' '9' '<EOS>']

Incorrect Sequence 5:
['0' '1' '1' '2' '3' '3' '3' '5' '6' '6' '6' '7']
['0' '1' '1' '2' '3' '3' '3' '3' '5' '6' '6' '6' '7']
Source:              ['6' '1' '5

 25%|██▌       | 4/16 [00:00<00:02,  5.78it/s]

...

Incorrect Sequence 48:
['0' '1' '1' '3' '3' '3' '3' '3' '4' '4' '5' '5' '6' '6' '8' '8' '9' '9'
 '9' '9']
['0' '1' '1' '3' '3' '3' '3' '3' '3' '4' '4' '5' '5' '6' '6' '8' '8' '9'
 '9' '9' '9']
Source:              ['9' '3' '8' '9' '9' '3' '4' '6' '0' '5' '3' '8' '3' '1' '5' '9' '3' '1'
 '6' '4']
Predicted Target:    ['<SOS>' '0' '1' '1' '3' '3' '3' '3' '3' '3' '4' '4' '5' '5' '6' '6' '8'
 '8' '9' '9' '9' '9' '<EOS>']

Incorrect Sequence 49:
['1' '3' '3' '3' '5' '6' '6' '7']
['1' '3' '3' '3' '3' '5' '6' '6' '7']
Source:              ['6' '3' '3' '1' '5' '6' '7' '3' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '1' '3' '3' '3' '3' '5' '6' '6' '7' '<EOS>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>']

Incorrect Sequence 50:
['1' '1' '1' '2' '2' '2' '2' '3' '3' '3' '5' '5' '6' '9' '9' '9' '9']
['1' '1' '1' '2' '2' '2' '2' '3' '3' '3' '3' '5' '5' '6' '9' '9'

 88%|████████▊ | 14/16 [00:02<00:00,  5.99it/s]

Incorrect Sequence 58:
['2' '3' '3' '3' '5' '5' '5' '6' '6' '7' '8' '9']
['2' '3' '3' '3' '3' '5' '5' '5' '6' '6' '7' '8' '9']
Source:              ['6' '8' '3' '9' '2' '5' '3' '7' '6' '5' '5' '3' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '2' '3' '3' '3' '3' '5' '5' '5' '6' '6' '7' '8' '9' '<EOS>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']

Incorrect Sequence 59:
['1' '3' '3' '3' '5' '6' '6' '7' '7' '7' '8' '9' '9']
['1' '3' '3' '3' '3' '5' '6' '6' '7' '7' '7' '8' '9' '9']
Source:              ['3' '7' '6' '3' '7' '6' '8' '3' '1' '9' '5' '9' '7' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '1' '3' '3' '3' '3' '5' '6' '6' '7' '7' '7' '8' '9' '9' '<EOS>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']

Incorrect Sequence 60:
['1' '2' '2' '2' '3' '3' '3' '4' '6']
['1' '2' '2' '2' '3' '3' '3' '3' '4' '6']
Source:              ['2' '2' '4' '3' '3' '1' '6' '3' '2' '<PAD>' '<PAD>'

100%|██████████| 16/16 [00:02<00:00,  5.91it/s]

Incorrect Sequence 67:
['0' '1' '3' '3' '3' '5' '6' '7' '7' '7' '8' '9' '9']
['0' '1' '3' '3' '3' '3' '5' '6' '7' '7' '7' '8' '9' '9']
Source:              ['3' '9' '0' '5' '8' '1' '7' '3' '3' '9' '7' '6' '7' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '0' '1' '3' '3' '3' '3' '5' '6' '7' '7' '7' '8' '9' '9' '<EOS>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']

Incorrect Sequence 68:
['0' '0' '2' '3' '3' '3' '3' '3' '4' '5' '6' '6' '6' '6' '6' '7' '7' '8'
 '8' '8']
['0' '0' '2' '3' '3' '3' '3' '3' '3' '4' '5' '6' '6' '6' '6' '6' '7' '7'
 '8' '8' '8']
Source:              ['5' '0' '6' '6' '8' '4' '0' '6' '3' '2' '3' '3' '8' '6' '3' '6' '3' '7'
 '7' '8']
Predicted Target:    ['<SOS>' '0' '0' '2' '3' '3' '3' '3' '3' '3' '4' '5' '6' '6' '6' '6' '6'
 '7' '7' '8' '8' '8' '<EOS>']

Incorrect Sequence 69:
['2' '3' '3' '5' '6' '6' '6' '6' '7' '7' '9']
['2' '3' '3' '3' '5' '6' '6' '6' '6' '7' '7' '9']
Source:              ['7' '6' '2' '3' '